In [ ]:

import pandas as pd
from math import sin, cos, sqrt, atan2, radians

In [ ]:
df_ps=pd.read_csv("csv/PS_civils_tous_avec_coordonnees.csv", sep=";",dtype=str)
df_communes=pd.read_csv("csv/communes_avec_operateurs_et_couverture_fixe.csv", sep=";", dtype=str)

In [ ]:
df_ps=df_ps[~df_ps["Libellé Voie (coord. structure)"].isna()]

In [ ]:
df_ps['Libellé savoir-faire']=df_ps['Libellé savoir-faire'].fillna('')
df_ps['Code savoir-faire']=df_ps['Code savoir-faire'].fillna('')

FAIRE DEUX DATASETS ET LES REMETTRE ENSEMBLE CAR LES FILTRES RESPECTIFS SONT EXCLUSIFS


LES MEDECINS GENERALISTES LIBERAUX 

In [ ]:
df_medecins=df_ps[df_ps["Libellé profession"]=="Médecin"]
df_medecins=df_medecins[df_medecins['Libellé savoir-faire'].isin(('Qualifié en Médecine Générale','Spécialiste en Médecine Générale','Médecine Générale'))]
df_medecins=df_medecins[df_medecins['Libellé mode exercice']=="Lib,indép,artis,com"]

LES PHARMACIENS


In [ ]:
df_pharmaciens=df_ps[df_ps["Libellé profession"]=="Pharmacien"]
df_pharmaciens=df_pharmaciens[df_pharmaciens["Libellé secteur d\'activité"]=="Pharmacie d'officine"]

In [ ]:
df_ps=pd.concat([df_pharmaciens,df_medecins],ignore_index=True)

POUR CALCULER DES INTERVALLES IL FAUT DES NOMBRES

In [ ]:
df_communes["latitude_mairie"]=df_communes["latitude_mairie"].replace(".",",").astype('float64')
df_communes["longitude_mairie"]=df_communes["longitude_mairie"].replace(".",",").astype('float64')

df_ps['lat']=df_ps['lat'].astype("float64")
df_ps['lon']=df_ps['lon'].astype("float64")

RETOURNE LE NOMBRE DE PROFESSIONNEL-LE-S DE SANTE DONT L'EMPLACEMENT EST DANS LA ZONE DELIMITEE (LE "RAYON" EST PLUTOT UN CARRE)

In [ ]:
liste_infos_cartes = []
cp_a = 1
nb_a = df_communes.shape[0]
liste_distances=[]
nb_med_aux_alentours = []
nb_phar_aux_alentours = []
rayon = 25 # en kilomètres
for ia, ra in df_communes.iterrows():

    print(f"{ra.loc["code_insee"]} {ra.loc["nom_standard"]} {cp_a}/{nb_a} ({round(((cp_a/nb_a)*100),3)}%)")
    
    lat_m = ra.loc["latitude_mairie"]
    lon_m = ra.loc["longitude_mairie"]

    rayon_v=0.1/11.122983322960587*rayon
    rayon_h=0.1/7.87939981553392*rayon

    d=df_ps[(df_ps['lat']>(lat_m-rayon_v))& (df_ps['lat']<(lat_m+rayon_v))&(df_ps['lon']>(lon_m-rayon_h))& (df_ps['lon']<(lon_m+rayon_h))]
    
    print(f"Nombre de PS : {d.shape[0]}")
    
    nb_med = d[d['Libellé profession']=='Médecin'].shape[0]
    print(f"Nombre de médecins : {nb_med}")
    nb_med_aux_alentours.append(nb_med)

    nb_phar=d[d['Libellé profession']=='Pharmacien'].shape[0]
    print(f"Nombre de pharmaciens : {nb_phar}")
    nb_phar_aux_alentours.append(nb_phar)

    for index, row in d.iterrows():
        # Rayon de la Terre
        R = 6373.0

        latitude2= float(row.loc['lat'])
        longitude2=float(row.loc['lon'])
        lat1 = radians(lat_m)
        lon1 = radians(lon_m)
        lat2 = radians(latitude2)
        lon2 = radians(longitude2)

        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))

        distance = R * c # en kilomètres
        liste_distances.append([ra.loc["code_insee"],row.loc['Identifiant PP'], distance])
    cp_a+=1
df_communes.insert(loc=len(df_communes.columns), column='nb_med', value=nb_med_aux_alentours)
df_communes.insert(loc=len(df_communes.columns), column='nb_phar', value=nb_phar_aux_alentours)


In [ ]:
df_distances=pd.DataFrame(liste_distances, columns=['code_insee','Identifiant PP','distance_en_km'])

In [ ]:
df_distances.to_csv('csv/table_distances.csv', sep=";", index=False)
df_communes.to_csv("csv/communes_avec_operateurs_et_couverture_fixe_et_med_et_pharm_dans_perimetre.csv", sep=";", index=False)